In [81]:
from __future__ import print_function

import numpy as np
from keras.callbacks import Callback
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential
from numpy.random import choice

USE_SEQUENCES = False
USE_STATELESS_MODEL = False

# you can all the four possible combinations
# USE_SEQUENCES and USE_STATELESS_MODEL

max_len = 5
batch_size = 1

N_train = 4
N_test = 200

X_train = np.zeros((N_train, max_len))
X_test = np.zeros((N_test, max_len))

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

y_train = np.zeros((N_train, 1))
y_test = np.zeros((N_test, 1))

one_indexes = choice(a=N_train, size=N_train / 2, replace=False)
X_train[one_indexes, 0] = 1
y_train[one_indexes] = 1

one_indexes = choice(a=N_test, size=N_test / 2, replace=False)
X_test[one_indexes, 0] = 1
y_test[one_indexes] = 1

X_train shape: (4, 5)
X_test shape: (200, 5)


/Users/williamcai/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:31: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/williamcai/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [82]:
class ResetStatesCallback(Callback):
    def __init__(self):
        self.counter = 0

    def on_batch_begin(self, batch, logs={}):
        if self.counter % max_len == 0:
            self.model.reset_states()
        self.counter += 1

In [83]:
X_train = np.expand_dims(X_train, axis=2)  # input dim is 1. Timesteps is the sequence length.
X_test = np.expand_dims(X_test, axis=2)

print('sequences_x_train shape:', X_train.shape)
print('sequences_y_train shape:', y_train.shape)

print('sequences_x_test shape:', X_test.shape)
print('sequences_y_test shape:', y_test.shape)



sequences_x_train shape: (4, 5, 1)
sequences_y_train shape: (4, 1)
sequences_x_test shape: (200, 5, 1)
sequences_y_test shape: (200, 1)


In [84]:
# STATEFUL MODEL
print('Build STATEFUL model...')
model = Sequential()
model.add(LSTM(512,
               batch_input_shape=(1, 1, 2), return_sequences=False,
               stateful=True))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


Build STATEFUL model...


In [85]:
n_train = 1000
max_len = 8
x = np.zeros((n_train,len_sequence,2))
for i in range(x.shape[0]):
    for ii in range(x.shape[1]):
        for iii in range(x.shape[2]):
            if i % 2 == 0 and ii == 0: 
                x[i][ii][iii] = 1


In [86]:
y = np.zeros((x.shape[0] * x.shape[1],1))
for i in range(x.shape[0]):
    for ii in range(x.shape[1]):
        if i % 2 == 0:
            y[i * x.shape[1] + ii] = 1

In [87]:
print(x.shape)
print(y.shape)


(1000, 8, 2)
(8000, 1)


In [88]:
new_x = np.zeros((x.shape[0] * x.shape[1],x.shape[2],1))
for i in range(x.shape[0]):
    for ii in range(x.shape[1]):
        for iii in range(2):
            new_x[i * x.shape[1] + ii][iii][0] = x[i][ii][iii]

In [89]:
print(new_x.shape)
print(y.shape)

(8000, 2, 1)
(8000, 1)


In [90]:
for i in range(10):
    print (i)
    print(new_x[i])
    print(y[i])


0
[[ 1.]
 [ 1.]]
[ 1.]
1
[[ 0.]
 [ 0.]]
[ 1.]
2
[[ 0.]
 [ 0.]]
[ 1.]
3
[[ 0.]
 [ 0.]]
[ 1.]
4
[[ 0.]
 [ 0.]]
[ 1.]
5
[[ 0.]
 [ 0.]]
[ 1.]
6
[[ 0.]
 [ 0.]]
[ 1.]
7
[[ 0.]
 [ 0.]]
[ 1.]
8
[[ 0.]
 [ 0.]]
[ 0.]
9
[[ 0.]
 [ 0.]]
[ 0.]


In [ ]:
model.fit(new_x,
          y,
          callbacks=[ResetStatesCallback()],
      batch_size=1, nb_epoch=10,
          shuffle=False)


Epoch 1/10
8000/8000 [==============================] - 1132s - loss: 0.0068 - acc: 0.9978  
Epoch 2/10
6300/8000 [======================>.......] - ETA: 284s - loss: 9.5149e-07 - acc: 1.0000

In [ ]:
print('Train...')
for epoch in range(15):
    mean_tr_acc = []
    mean_tr_loss = []
    for i in range(len(X_train)):
        y_true = y_train[i]
        for j in range(max_len):
            tr_loss, tr_acc = model.train_on_batch(np.expand_dims(np.expand_dims(X_train[i][j], axis=1), axis=1),
                                                   np.array([y_true]))
            mean_tr_acc.append(tr_acc)
            mean_tr_loss.append(tr_loss)
        model.reset_states()

    print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
    print('loss training = {}'.format(np.mean(mean_tr_loss)))
    print('___________________________________')

    mean_te_acc = []
    mean_te_loss = []
    for i in range(len(X_test)):
        for j in range(max_len):
            te_loss, te_acc = model.test_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1),
                                                  y_test[i])
            mean_te_acc.append(te_acc)
            mean_te_loss.append(te_loss)
        model.reset_states()

        for j in range(max_len):
            y_pred = model.predict_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1))
        model.reset_states()

    print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
    print('loss testing = {}'.format(np.mean(mean_te_loss)))
    print('___________________________________')